In [11]:
import pandas as pd

In [12]:
#Load file with all the recipes
df = pd.read_excel("../data/raw/Recipes.xlsx")

In [13]:
df.head()

,Nombre,Tipo,Link
0,Vegetable Ramen with Mushrooms and Bok Choy,Veg,https://www.goodhousekeeping.com/food-recipes/...
1,Grilled Steak Tortilla Salad,Meat,https://www.goodhousekeeping.com/food-recipes/...
2,Cauliflower Pizza,Veg,https://www.goodhousekeeping.com/food-recipes/...
3,Honey-Ginger Cedar Plank Salmon,Fish,https://www.goodhousekeeping.com/food-recipes/...
4,Traditional Chicken Curry,Meat,https://www.goodhousekeeping.com/food-recipes/...


In [14]:
#Removing spaces
df.Tipo = df.Tipo.str.strip()

In [15]:
#Loading previous file
df_previous_recipes = pd.read_csv("../data/processed/previous_recipes.xlsx")
list_columns = list(df_previous_recipes.columns)
for column in list_columns:
    if column == "Unnamed: 0":
        df_previous_recipes = df_previous_recipes.drop("Unnamed: 0",axis=1)

In [16]:
#Removing previous recipes
df = df[~df.Nombre.isin(df_previous_recipes.Nombre.unique())]

In [17]:
#Selecting random recipes
df_veg = df[df.Tipo.str.lower() == "veg"].sample(n = 2)
df_meat = df[df.Tipo.str.lower()== "meat"].sample(n = 1)
df_fish = df[df.Tipo.str.lower()== "fish"].sample(n = 0)
df_pasta = df[df.Tipo.str.lower() == "pasta"].sample(n = 1)

In [18]:
#putting together recipes
df_final = pd.concat([pd.concat([df_veg, df_meat]), pd.concat([df_fish, df_pasta])]).reset_index(drop=True)

In [19]:
df_final

,Nombre,Tipo,Link
0,Risotto zafferano,Veg,Saffron risotto recipe | BBC Good Food
1,Gnocchi al pesto,Veg,https://littlesunnykitchen.com/creamy-pesto-gn...
2,Taco Pie,Meat,Easy Taco Pie Recipe | Life Tastes Good (ourli...
3,Wholewheat spaghetti & avocado sauce,Pasta,https://www.bbcgoodfood.com/recipes/wholewheat...


In [20]:
#Loading data from recipes
pd.concat([df_previous_recipes,df_final]).reset_index(drop=True).to_csv("../data/processed/previous_recipes.xlsx")

# Sending EMAIL

In [21]:
#Creating subject of mail
from datetime import datetime
year, week_num, day_of_week = datetime.today().isocalendar()
subject = "Menu Week " + str(week_num) +" " + str(year)

In [22]:
df_final

,Nombre,Tipo,Link
0,Risotto zafferano,Veg,Saffron risotto recipe | BBC Good Food
1,Gnocchi al pesto,Veg,https://littlesunnykitchen.com/creamy-pesto-gn...
2,Taco Pie,Meat,Easy Taco Pie Recipe | Life Tastes Good (ourli...
3,Wholewheat spaghetti & avocado sauce,Pasta,https://www.bbcgoodfood.com/recipes/wholewheat...


In [23]:
body = ""
for i in range(0,df_final.shape[0]):
    body = body + df_final.loc[i, 'Nombre'] + ": " + df_final.loc[i, 'Link'] + "\n"
print(body)

Risotto zafferano : Saffron risotto recipe | BBC Good Food
Gnocchi al pesto: https://littlesunnykitchen.com/creamy-pesto-gnocchi/
Taco Pie: Easy Taco Pie Recipe | Life Tastes Good (ourlifetastesgood.com)
Wholewheat spaghetti & avocado sauce: https://www.bbcgoodfood.com/recipes/wholewheat-spaghetti-avocado-sauce



In [24]:
import smtplib

gmail_user = 'sabaku88@gmail.com'
gmail_password = 'ucalhamhnnyyhmqz'

sent_from = gmail_user
to = ['sabaku88@gmail.com', 'chiara.tronconi.91@gmail.com']
#subject = 'OMG Super Important Message'
#body = str(df_final.iloc[0].Nombre) + ": " + str(df_final.iloc[0].Link)

email_text = """\
From: %s
To: %s
Subject: %s

%s
""" % (sent_from, ", ".join(to), subject, body)

try:
    server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
    server.ehlo()
    server.login(gmail_user, gmail_password)
    server.sendmail(sent_from, to, email_text)
    server.close()

    print('Email sent!')
except:
    print('Something went wrong...')

Email sent!
